## Mini Batch / Minibatch Gradient Descent / Data Load / Custom Dataset

### Mini Batch and Batch Size

데이터가 수십만개 이상이라면 전체 데이터에 대해서 경사 하강법을 수행하는 것은 매우 느릴 뿐만 아니라 많은 계산량이 필요

`Mini Batch` : 전체 데이터를 더 작은 단위로 나누어서 학습하는 단위

미니 배치 학습을 하게되면 미니 배치만큼만 가져가서 미니 배치에 대한 대한 cost를 계산하고, 경사 하강법을 수행

다음 미니 배치를 가져가서 경사 하강법을 수행하고 마지막 미니 배치까지 이를 반복

전체 데이터에 대한 학습이 1회 끝나면 1 Epoch(에폭)이 끝남

`Epoch` : 전체 훈련 데이터가 학습에 한 번 사용된 주기

미니 배치 학습에서는 미니 배치의 개수만큼 경사 하강법을 수행해야 전체 데이터가 한 번 전부 사용되어 1 에포크(Epoch)가 됨

미니 배치의 개수는 결국 미니 배치의 크기를 몇으로 하느냐에 따라서 달라짐

`batch size` : mini batch의 크기

`batch gradient descent` : 전체 데이터에 대해서 한 번에 경사 하강법을 수행하는 방법, 전체 데이터를 사용하므로 가중치 값이 최적값에 수렴하는 과정이 매우 안정적, 계산량이 너무 많음

`minibatch gradient descent` : 미니 배치 단위로 경사 하강법을 수행하는 방법, 전체 데이터의 일부만을 보고 수행하므로 최적값으로 수렴하는데 헤맴, 훈련 속도가 빠름

배치 크기는 보통 2의 제곱수를 사용, CPU와 GPU의 메모리가 2의 배수이므로 배치크기가 2의 제곱수일 경우에 데이터 송수신의 효율을 높일 수 있음

### Iteration(이터레이션)

`Iteration` : 한 번의 Epoch 내에서 이루어지는 매개변수인 가중치 W와 b의 업데이트 횟수

전체 데이터가 2000이고 배치 크기가 200이면 이터레이션은 10

에폭당 매겨변수 업데이트가 10번 이루어짐을 의미

### Data Load

파이토치에서는 데이터를 좀 더 쉽게 다룰 수 있도록 유용한 도구로서 데이터셋(Dataset)과 데이터로더(DataLoader)를 제공

미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리 기능이 포함됨

기본적인 사용 방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [51]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [52]:
# TensorDataset은 기본적으로 텐서를 입력으로 받습니다. 텐서 형태로 데이터를 정의
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])
dataset = TensorDataset(x_train, y_train)

파이토치의 데이터셋을 만들었다면 데이터로더를 사용 가능

데이터로더는 기본적으로 2개의 인자를 입력, 데이터셋, 미니배치 크기

많이 사용되는 인자로 shuffle이 있음 shuffle=True를 선택하면 Epoch마다 데이터셋을 섞어서 데이터가 학습되는 순서를 바꿈

모델이 데이터셋의 순서에 익숙해지는 것을 방지하여 학습할 때는 이 옵션을 True를 주는 것을 권장

In [53]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [54]:
model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

In [55]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        print(batch_idx)
        print(samples)
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print(f'Epoch {epoch:4d}/{nb_epochs} Batch {batch_idx+1}/{len(dataloader)} Cost: {cost.item():.6f}')

0
[tensor([[93., 88., 93.],
        [89., 91., 90.]]), tensor([[185.],
        [180.]])]
Epoch    0/20 Batch 1/3 Cost: 50238.281250
1
[tensor([[73., 66., 70.],
        [73., 80., 75.]]), tensor([[142.],
        [152.]])]
Epoch    0/20 Batch 2/3 Cost: 8322.404297
2
[tensor([[ 96.,  98., 100.]]), tensor([[196.]])]
Epoch    0/20 Batch 3/3 Cost: 6578.057617
0
[tensor([[93., 88., 93.],
        [73., 66., 70.]]), tensor([[185.],
        [142.]])]
Epoch    1/20 Batch 1/3 Cost: 1125.345093
1
[tensor([[ 73.,  80.,  75.],
        [ 96.,  98., 100.]]), tensor([[152.],
        [196.]])]
Epoch    1/20 Batch 2/3 Cost: 245.946228
2
[tensor([[89., 91., 90.]]), tensor([[180.]])]
Epoch    1/20 Batch 3/3 Cost: 89.446121
0
[tensor([[89., 91., 90.],
        [73., 66., 70.]]), tensor([[180.],
        [142.]])]
Epoch    2/20 Batch 1/3 Cost: 55.478935
1
[tensor([[ 93.,  88.,  93.],
        [ 96.,  98., 100.]]), tensor([[185.],
        [196.]])]
Epoch    2/20 Batch 2/3 Cost: 24.054092
2
[tensor([[73., 80., 75.

In [56]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print(pred_y)

tensor([[156.0985]], grad_fn=<AddmmBackward0>)


### Custom Dataset

torch.utils.data.Dataset을 상속받아 직접 커스텀 데이터셋(Custom Dataset)을 만들 수 있음

torch.utils.data.Dataset은 파이토치에서 데이터셋을 제공하는 추상 클래스

가장 기본적인 define은 3개로 init, len, getitem임

init은 생성자로  데이터셋의 전처리를 해주는 부분

len은 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분

getitem은 데이터셋에서 특정 1개의 샘플을 가져오는 매서드로 인덱싱에 사용되는 부분

In [57]:
from torch.utils.data import Dataset

In [58]:
# Dataset 상속
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]

    # 총 데이터의 개수를 리턴
    def __len__(self):
        return len(self.x_data)
    
    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

In [59]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [60]:
model = torch.nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

In [61]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        #print(batch_idx)
        #print(samples)
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print(f'Epoch {epoch:4d}/{nb_epochs} Batch {batch_idx+1}/{len(dataloader)} Cost: {cost.item():.6f}')

Epoch    0/20 Batch 1/3 Cost: 16755.916016
Epoch    0/20 Batch 2/3 Cost: 11733.840820
Epoch    0/20 Batch 3/3 Cost: 3416.828369
Epoch    1/20 Batch 1/3 Cost: 480.786804
Epoch    1/20 Batch 2/3 Cost: 184.408417
Epoch    1/20 Batch 3/3 Cost: 55.141773
Epoch    2/20 Batch 1/3 Cost: 12.953405
Epoch    2/20 Batch 2/3 Cost: 3.665296
Epoch    2/20 Batch 3/3 Cost: 7.415599
Epoch    3/20 Batch 1/3 Cost: 1.587905
Epoch    3/20 Batch 2/3 Cost: 0.689919
Epoch    3/20 Batch 3/3 Cost: 0.001479
Epoch    4/20 Batch 1/3 Cost: 1.352613
Epoch    4/20 Batch 2/3 Cost: 0.089144
Epoch    4/20 Batch 3/3 Cost: 1.423597
Epoch    5/20 Batch 1/3 Cost: 0.320588
Epoch    5/20 Batch 2/3 Cost: 1.540054
Epoch    5/20 Batch 3/3 Cost: 1.357885
Epoch    6/20 Batch 1/3 Cost: 0.311991
Epoch    6/20 Batch 2/3 Cost: 1.576253
Epoch    6/20 Batch 3/3 Cost: 1.364614
Epoch    7/20 Batch 1/3 Cost: 0.329488
Epoch    7/20 Batch 2/3 Cost: 1.819390
Epoch    7/20 Batch 3/3 Cost: 0.004660
Epoch    8/20 Batch 1/3 Cost: 1.356111
Epoch   

In [62]:
new_var =  torch.FloatTensor([[73, 80, 75]]) 
pred_y = model(new_var) 
print(pred_y)

tensor([[150.0981]], grad_fn=<AddmmBackward0>)
